### Preprocessing

In [3]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

#  Import and read the charity_data.csv.
import pandas as pd 
vax_df = pd.read_csv("sample_data/New_York_State_Statewide_COVID-19_Vaccination_Data_by_County.csv")
demo_df = pd.read_csv("sample_data/Full_2020PL_noblocks.csv")
vax_df.head()
demo_df.head()

,Region,County,First Dose,Series Complete,Report as of
0,Capital Region,Albany,69,0,2020-12-14
1,Western New York,Allegany,0,0,2020-12-14
2,New York City,Bronx,2,0,2020-12-14
3,Southern Tier,Broome,0,0,2020-12-14
4,Western New York,Cattaraugus,0,0,2020-12-14


In [4]:
# Drop rows unneccesary columns
columns_to_drop = ['COUNTY','FILEID', 'STUSAB', 'SUMLEV', 'GEOVAR', 'GEOCOMP', 'CHARITER', 'CIFSN', 'LOGRECNO', 'GEOID', 'GEOCODE', 'REGION', 'DIVISION', 'STATE', 'STATENS', 'COUNTYCC', 'COUNTYNS', 'COUSUB', 'COUSUBCC', 'COUSUBNS', 'SUBMCD', 'SUBMCDCC', 'SUBMCDNS', 'ESTATE', 'ESTATECC', 'ESTATENS', 'CONCIT', 'CONCITCC', 'CONCITNS', 'PLACE', 'PLACECC', 'PLACENS', 'TRACT', 'BLKGRP', 'BLOCK', 'AIANHH', 'AIHHTLI', 'AIANHHFP', 'AIANHHCC', 'AIANHHNS', 'AITS', 'AITSFP', 'AITSCC', 'AITSNS', 'TTRACT', 'TBLKGRP', 'ANRC', 'ANRCCC', 'ANRCNS', 'CBSA', 'MEMI', 'CSA', 'METDIV', 'NECTA', 'NMEMI', 'CNECTA', 'NECTADIV', 'CBSAPCI', 'NECTAPCI', 'UA', 'UATYPE', 'UR', 'CD116', 'CD118', 'CD119', 'CD120', 'CD121', 'SLDU18', 'SLDU22', 'SLDU24', 'SLDU26', 'SLDU28', 'SLDL18', 'SLDL22', 'SLDL24', 'SLDL26', 'SLDL28', 'VTD', 'VTDI', 'ZCTA', 'SDELM', 'SDSEC', 'SDUNI', 'PUMA', 'AREALAND', 'AREAWATR', 'NAME', 'FUNCSTAT', 'GCUNI', 'HU100', 'INTPTLAT', 'INTPTLON', 'LSADC', 'PARTFLAG', 'UGA']
demo_df = demo_df.drop(columns_to_drop, axis=1)

# Rename county column
demo_df.rename(columns={'BASENAME': 'County'}, inplace=True)

# Drop rows with NA
demo_df = demo_df.dropna()
demo_df = demo_df.loc[demo_df['County'] != 'County subdivisions not defined']

demo_df


,County,POP100,P0010001,P0010002,P0010003,P0010004,P0010005,P0010006,P0010007,P0010008,P0010009,P0010010,P0010011,P0010012,P0010013,P0010014,P0010015,P0010016,P0010017,P0010018,P0010019,P0010020,P0010021,P0010022,P0010023,P0010024,P0010025,P0010026,P0010027,P0010028,P0010029,P0010030,P0010031,P0010032,P0010033,P0010034,P0010035,P0010036,P0010037,P0010038,...,P0020034,P0020035,P0020036,P0020037,P0020038,P0020039,P0020040,P0020041,P0020042,P0020043,P0020044,P0020045,P0020046,P0020047,P0020048,P0020049,P0020050,P0020051,P0020052,P0020053,P0020054,P0020055,P0020056,P0020057,P0020058,P0020059,P0020060,P0020061,P0020062,P0020063,P0020064,P0020065,P0020066,P0020067,P0020068,P0020069,P0020070,P0020071,P0020072,P0020073
0,Albany,314848,314848,293076,215496,43206,845,24473,183,8873,21772,20193,4611,2945,2493,70,7332,561,193,37,1633,25,3,97,54,94,45,1410,525,126,16,355,26,4,172,54,42,5,9,0,...,4,14,47,18,0,6,0,12,0,11,4,5,7,0,0,60,27,14,14,0,1,0,2,1,0,0,0,1,0,0,0,13,9,4,0,0,0,0,0,0
1,Allegany,46456,46456,44541,42555,829,91,379,9,678,1915,1824,220,693,138,17,661,12,8,0,30,8,0,0,32,1,4,82,20,7,0,4,5,1,18,10,7,0,1,0,...,1,4,9,3,0,0,0,0,0,2,0,0,0,0,2,4,3,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,0,2,0,0,0,0,0,0
2,Bronx,1472654,1472654,1280481,207573,487118,21539,69559,1640,493052,192173,175249,12298,1667,3877,110,77946,4766,1798,478,59626,496,103,6351,382,3084,2267,13624,2698,417,36,6508,68,15,1409,66,401,75,122,23,...,10,8,49,62,3,88,14,60,36,117,17,7,35,3,19,285,104,15,81,4,34,0,2,3,3,13,2,6,1,13,4,53,24,29,0,0,0,0,4,4
3,Broome,198683,198683,185345,158674,12684,556,9372,84,3975,13338,12412,3053,2685,1162,70,4411,266,89,50,513,7,0,23,27,42,14,813,330,61,15,167,42,8,71,33,13,7,7,5,...,1,13,28,10,4,6,0,4,5,10,1,0,0,3,0,30,17,3,3,0,6,0,0,0,0,0,0,0,1,0,0,4,3,0,1,0,0,0,0,0
4,Cattaraugus,77042,77042,73029,68128,1043,2706,567,14,571,4013,3789,689,1547,189,12,1233,36,4,1,39,8,3,10,10,0,8,172,70,2,0,20,8,2,17,20,12,1,1,0,...,0,9,16,6,0,1,0,0,0,0,2,0,0,0,0,13,6,3,3,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1895,Allegany,46456,46456,44541,42555,829,91,379,9,678,1915,1824,220,693,138,17,661,12,8,0,30,8,0,0,32,1,4,82,20,7,0,4,5,1,18,10,7,0,1,0,...,1,4,9,3,0,0,0,0,0,2,0,0,0,0,2,4,3,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,0,2,0,0,0,0,0,0
1896,Cattaraugus,77042,77042,73029,68128,1043,2706,567,14,571,4013,3789,689,1547,189,12,1233,36,4,1,39,8,3,10,10,0,8,172,70,2,0,20,8,2,17,20,12,1,1,0,...,0,9,16,6,0,1,0,0,0,0,2,0,0,0,0,13,6,3,3,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0
1897,Steuben,3555,3555,3473,3445,9,4,1,0,14,82,79,8,30,4,1,36,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1898,Erie,137195,137195,127478,96518,17209,882,5707,55,7107,9717,9134,1651,1167,817,32,4453,187,71,16,552,14,3,46,30,69,26,512,176,28,5,148,9,1,64,21,18,3,0,0,...,1,0,20,6,0,0,0,2,1,0,0,0,0,0,0,21,12,0,3,0,0,0,0,1,0,1,3,0,1,0,0,2,2,0,0,0,0,0,0,0


In [5]:
#Remove unnecessary columns
vax_df = vax_df.drop(['Region', 'First Dose'], axis=1)

# Keep most recent dates
vax_df = pd.DataFrame(vax_df.loc[vax_df['Report as of'] == '2021-10-22'])

vax_df


,County,Series Complete,Report as of
19344,Albany,204365,2021-10-22
19345,Allegany,18673,2021-10-22
19346,Bronx,857631,2021-10-22
19347,Broome,113433,2021-10-22
19348,Cattaraugus,34875,2021-10-22
...,...,...,...
19401,Washington,35067,2021-10-22
19402,Wayne,53418,2021-10-22
19403,Westchester,671155,2021-10-22
19404,Wyoming,19245,2021-10-22


In [6]:
# join files
df = demo_df.merge(vax_df, how='inner',on='County')
df.head()

,County,POP100,P0010001,P0010002,P0010003,P0010004,P0010005,P0010006,P0010007,P0010008,P0010009,P0010010,P0010011,P0010012,P0010013,P0010014,P0010015,P0010016,P0010017,P0010018,P0010019,P0010020,P0010021,P0010022,P0010023,P0010024,P0010025,P0010026,P0010027,P0010028,P0010029,P0010030,P0010031,P0010032,P0010033,P0010034,P0010035,P0010036,P0010037,P0010038,...,P0020036,P0020037,P0020038,P0020039,P0020040,P0020041,P0020042,P0020043,P0020044,P0020045,P0020046,P0020047,P0020048,P0020049,P0020050,P0020051,P0020052,P0020053,P0020054,P0020055,P0020056,P0020057,P0020058,P0020059,P0020060,P0020061,P0020062,P0020063,P0020064,P0020065,P0020066,P0020067,P0020068,P0020069,P0020070,P0020071,P0020072,P0020073,Series Complete,Report as of
0,Albany,314848,314848,293076,215496,43206,845,24473,183,8873,21772,20193,4611,2945,2493,70,7332,561,193,37,1633,25,3,97,54,94,45,1410,525,126,16,355,26,4,172,54,42,5,9,0,...,47,18,0,6,0,12,0,11,4,5,7,0,0,60,27,14,14,0,1,0,2,1,0,0,0,1,0,0,0,13,9,4,0,0,0,0,0,0,204365,2021-10-22
1,Albany,99224,99224,91331,46391,31023,436,7993,77,5411,7893,7212,2001,570,822,13,2019,442,83,25,1101,20,2,57,15,29,13,606,221,54,8,183,10,1,52,23,19,0,5,0,...,19,7,0,3,0,11,0,3,1,1,2,0,0,21,9,4,5,0,1,0,0,1,0,0,0,1,0,0,0,6,4,2,0,0,0,0,0,0,204365,2021-10-22
2,Albany,1675,1675,1565,1527,14,3,13,2,6,110,102,25,22,8,0,44,2,1,0,0,0,0,0,0,0,0,7,4,0,0,2,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,204365,2021-10-22
3,Albany,18147,18147,16689,14327,1507,51,423,2,379,1458,1328,408,295,108,4,395,21,11,1,79,0,0,5,0,0,1,124,71,14,0,19,2,0,11,5,1,1,0,0,...,5,1,0,0,0,0,0,0,0,0,0,0,0,2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,204365,2021-10-22
4,Albany,7781,7781,7276,5891,516,10,738,0,121,505,471,98,90,40,4,198,7,9,1,17,0,0,2,1,2,2,32,15,5,0,6,0,0,5,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,204365,2021-10-22


In [7]:
# Calculate Vax Rate
df['Vax_Rate'] = (df['Series Complete'] / df['POP100']) * 100
df['Vax_Rate'] = df['Vax_Rate'].round()
df['Vax_Rate'] = df['Vax_Rate'].mask(df['Vax_Rate']>100,100).astype('int64')


In [8]:
df['Vax_Rate'].unique()

array([ 65, 100,  81,  40,  58,  57,  85,  45,  54,  80,  51,  48,  55,
        79,  72,  63,  61,  53,  82,  50,  90,  99,  60,  86,  62,  46,
        56,  98,  67,  59,  70,  71,  92,  88,  83,  49,  89,  66,  91,
        64,  87,  69,  93,  52,  97,  47])

In [9]:
# Drop the non-beneficial columns, County, First Dose, Series Complete, Report as of, Location
df = df.drop(['County', 'Report as of'], axis=1)

df.head()

,POP100,P0010001,P0010002,P0010003,P0010004,P0010005,P0010006,P0010007,P0010008,P0010009,P0010010,P0010011,P0010012,P0010013,P0010014,P0010015,P0010016,P0010017,P0010018,P0010019,P0010020,P0010021,P0010022,P0010023,P0010024,P0010025,P0010026,P0010027,P0010028,P0010029,P0010030,P0010031,P0010032,P0010033,P0010034,P0010035,P0010036,P0010037,P0010038,P0010039,...,P0020036,P0020037,P0020038,P0020039,P0020040,P0020041,P0020042,P0020043,P0020044,P0020045,P0020046,P0020047,P0020048,P0020049,P0020050,P0020051,P0020052,P0020053,P0020054,P0020055,P0020056,P0020057,P0020058,P0020059,P0020060,P0020061,P0020062,P0020063,P0020064,P0020065,P0020066,P0020067,P0020068,P0020069,P0020070,P0020071,P0020072,P0020073,Series Complete,Vax_Rate
0,314848,314848,293076,215496,43206,845,24473,183,8873,21772,20193,4611,2945,2493,70,7332,561,193,37,1633,25,3,97,54,94,45,1410,525,126,16,355,26,4,172,54,42,5,9,0,37,...,47,18,0,6,0,12,0,11,4,5,7,0,0,60,27,14,14,0,1,0,2,1,0,0,0,1,0,0,0,13,9,4,0,0,0,0,0,0,204365,65
1,99224,99224,91331,46391,31023,436,7993,77,5411,7893,7212,2001,570,822,13,2019,442,83,25,1101,20,2,57,15,29,13,606,221,54,8,183,10,1,52,23,19,0,5,0,19,...,19,7,0,3,0,11,0,3,1,1,2,0,0,21,9,4,5,0,1,0,0,1,0,0,0,1,0,0,0,6,4,2,0,0,0,0,0,0,204365,100
2,1675,1675,1565,1527,14,3,13,2,6,110,102,25,22,8,0,44,2,1,0,0,0,0,0,0,0,0,7,4,0,0,2,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,204365,100
3,18147,18147,16689,14327,1507,51,423,2,379,1458,1328,408,295,108,4,395,21,11,1,79,0,0,5,0,0,1,124,71,14,0,19,2,0,11,5,1,1,0,0,0,...,5,1,0,0,0,0,0,0,0,0,0,0,0,2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,204365,100
4,7781,7781,7276,5891,516,10,738,0,121,505,471,98,90,40,4,198,7,9,1,17,0,0,2,1,2,2,32,15,5,0,6,0,0,5,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,204365,100


In [10]:
# Check all values are numeric
df.info()



<class 'pandas.core.frame.DataFrame'>
Int64Index: 1870 entries, 0 to 1869
Columns: 147 entries, POP100 to Vax_Rate
dtypes: int64(147)
memory usage: 2.1 MB


In [11]:
# Split our preprocessed data into our features and target arrays
y = df["Vax_Rate"].values
X = df.drop(["Vax_Rate"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [12]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

### Linear Regression

In [13]:
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [14]:
y_pred = model.predict(X_test)
print(y_pred.shape)

(468,)


In [16]:
print(model.coef_)
print(model.intercept_)

[ 1.47617300e+05  4.29436609e+07  2.15605760e+07  4.26237919e+06
  4.26237919e+06  4.26237920e+06  4.26237927e+06  4.26237925e+06
  4.26237920e+06  2.11084905e+07  4.29448201e+06  4.19982744e+05
  4.19982731e+05  4.19982628e+05  4.19983122e+05  4.19982693e+05
  4.19982678e+05  4.19983110e+05  4.19982146e+05  4.19982700e+05
  4.19982593e+05  4.19982125e+05  4.19982712e+05  4.19981638e+05
  4.19982677e+05  4.19982672e+05  4.48292051e+06  2.31544134e+05
  2.31544194e+05  2.31542713e+05  2.31544104e+05  2.31543920e+05
  2.31542393e+05  2.31544112e+05  2.31541941e+05  2.31544196e+05
  2.31542989e+05  2.31543264e+05  2.31545714e+05  2.31544253e+05
  2.31543197e+05  2.31543952e+05  2.31544155e+05  2.31547466e+05
  2.31544334e+05  2.31543597e+05  2.31543165e+05  4.42158146e+06
  2.92883602e+05  2.92883461e+05  2.92883410e+05  2.92881829e+05
  2.92883092e+05  2.92882803e+05  2.92883050e+05  2.92883274e+05
  2.92880623e+05  2.92883586e+05  2.92885475e+05  2.92885631e+05
  2.92881035e+05  2.92884

### Neural Network

In [17]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 20
hidden_nodes_layer2 = 20

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, 
                          input_dim=number_input_features, 
                          activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, 
                             activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="linear"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 20)                2940      
_________________________________________________________________
dense_1 (Dense)              (None, 20)                420       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 21        
Total params: 3,381
Trainable params: 3,381
Non-trainable params: 0
_________________________________________________________________


In [18]:
# Compile the model
nn.compile(loss="mean_absolute_error", 
           optimizer="adam")

In [19]:
# Create a callback that saves the model's weights every 5 epochs
callbacks = tf.keras.callbacks.ModelCheckpoint('weights{epoch:08d}.h5', 
                                     save_weights_only=True, save_freq=5)

In [20]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100, callbacks=[callbacks])

Epoch 1/100
44/44 [==============================] - 1s 3ms/step - loss: 94.1992
Epoch 2/100
44/44 [==============================] - 0s 3ms/step - loss: 87.0430
Epoch 3/100
44/44 [==============================] - 0s 3ms/step - loss: 64.6985
Epoch 4/100
44/44 [==============================] - 0s 3ms/step - loss: 27.2337
Epoch 5/100
44/44 [==============================] - 0s 6ms/step - loss: 21.5322
Epoch 6/100
44/44 [==============================] - 0s 3ms/step - loss: 20.8816
Epoch 7/100
44/44 [==============================] - 0s 3ms/step - loss: 20.1555
Epoch 8/100
44/44 [==============================] - 0s 3ms/step - loss: 19.6127
Epoch 9/100
44/44 [==============================] - 0s 3ms/step - loss: 18.9433
Epoch 10/100
44/44 [==============================] - 0s 4ms/step - loss: 18.5220
Epoch 11/100
44/44 [==============================] - 0s 3ms/step - loss: 17.9824
Epoch 12/100
44/44 [==============================] - 0s 3ms/step - loss: 17.3540
Epoch 13/100
44/44 [=====

In [21]:
# Evaluate the model using the test data
model_loss = nn.evaluate(X_test_scaled,y_test,verbose=1)
print(f"Loss: {model_loss}")

15/15 [==============================] - 0s 1ms/step - loss: 4.5200
Loss: 4.52000093460083


In [22]:
#Save 
nn.save('COVID_Vax_Predictions.h5')